In [44]:
import branca
import csv
import datetime
import folium
import sqlite3
import ipywidgets as widgets
import numpy as np
import pandas as pd
import draw_markers
import table_builder


In [2]:
table_builder.create_table("my_db","data/")

In [3]:
connection = sqlite3.connect("my_db")
crsr = connection.cursor()


In [16]:
crsr.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = (crsr.fetchall())

print(tables)
crsr.execute("select * from buildings")
buildings_fields = list(map(lambda x: x[0], crsr.description))

crsr.execute("select * from collisions")
collisions_fields = list(map(lambda x: x[0], crsr.description))

crsr.execute("select * FROM Collision_count_BDA_Const_all")
radius_fields = list(map(lambda x: x[0], crsr.description))


[('buildings',), ('collisions',), ('Collision_count_BDA_Const_all',), ('Coll_Within_One_Year_of_Const',), ('radius',)]


In [17]:
print("building fields:", buildings_fields)
print("collisions fields:", collisions_fields)
print("radius fields:", radius_fields)

building fields: ['text', 'b_id', 'b_category', 'b_value', 'b_issue_date', 'b_final_date', 'b_status', 'b_lat', 'b_long']
collisions fields: ['text', 'c_id', 'c_long', 'c_lat', 'c_datetime', 'c_ped', 'c_cyc', 'c_severity_code', 'c_severity_desc', 'c_veh']
radius fields: ['text', 'build_end_dt', 'build_id', 'build_lat', 'build_long', 'build_start_dt', 'coll_dt', 'coll_id', 'coll_lat', 'coll_long', 'radius', 'coll_before', 'coll_during', 'coll_after']


In [45]:
select_query = """SELECT b.b_id as b_id, 
    b.b_lat as b_latitude,
    b.b_long as b_longitude, 
    sum(coll_before) AS col_before, 
    sum(coll_during) AS col_during, 
    sum(coll_after) AS col_after """
from_query = """FROM buildings b, Collision_count_BDA_Const_all r """
where_query = """WHERE b.b_id = r.build_id """
group_by_query = """GROUP BY b.b_id"""
whole_query = str(select_query + from_query + where_query + group_by_query)

crsr.execute(whole_query)
test = pd.DataFrame(crsr.fetchall())
test.columns = list(map(lambda x: x[0], crsr.description))
test['b_latitude'] = test.b_latitude.astype(float)
test['b_longitude'] = test.b_longitude.astype(float)


In [84]:
def build_type_interact(Building_Category):
    
    if Building_Category == "All":
         where_query = "WHERE b.b_id = r.build_id "
    else:
        where_query = str("""WHERE b.b_id = r.build_id and b.b_category = """ + str("'" + Building_Category + "'"))
    select_query = """SELECT b.b_id as b_id, 
    b.b_lat as b_latitude,
    b.b_long as b_longitude, 
    sum(coll_before) AS col_before, 
    sum(coll_during) AS col_during, 
    sum(coll_after) AS col_after """
    from_query = """FROM buildings b, Collision_count_BDA_Const_all r """
    group_by_query = """ GROUP BY b.b_id"""
    whole_query = str(select_query + from_query + where_query + group_by_query)
#     print(whole_query)
    crsr.execute(whole_query)
    temp = pd.DataFrame(crsr.fetchall())
    temp.columns = list(map(lambda x: x[0], crsr.description))
    temp['b_latitude'] = test.b_latitude.astype(float)
    temp['b_longitude'] = test.b_longitude.astype(float)
    return draw_markers.place_maps(temp)

categories = ['All','COMMERCIAL','MULTIFAMILY', 'INDUSTRIAL','INSTITUTIONAL','SINGLE FAMILY / DUPLEX']


In [85]:
widgets.interact(build_type_interact, Building_Category = categories)

interactive(children=(Dropdown(description='Building_Category', options=('All', 'COMMERCIAL', 'MULTIFAMILY', '…

<function __main__.build_type_interact(Building_Category)>

In [65]:
def build_type_interact(Building_Category):
    
    if Building_Category == "All":
         where_query = "WHERE b.b_id = r.build_id "
    else:
        where_query = str("""WHERE b.b_id = r.build_id and b.b_category = """ + str("'" + Building_Category + "'"))
    select_query = """SELECT b.b_id as b_id, 
    b.b_lat as b_latitude,
    b.b_long as b_longitude, 
    sum(coll_before) AS col_before, 
    sum(coll_during) AS col_during, 
    sum(coll_after) AS col_after """
    from_query = """FROM buildings b, Collision_count_BDA_Const_all r """
    group_by_query = """ GROUP BY b.b_id"""
    whole_query = str(select_query + from_query + where_query + group_by_query)
#     print(whole_query)
    crsr.execute(whole_query)
    temp = pd.DataFrame(crsr.fetchall())
    temp.columns = list(map(lambda x: x[0], crsr.description))
    temp['b_latitude'] = test.b_latitude.astype(float)
    temp['b_longitude'] = test.b_longitude.astype(float)
    return draw_markers.place_maps(temp)

categories = ['All','COMMERCIAL','MULTIFAMILY', 'INDUSTRIAL','INSTITUTIONAL','SINGLE FAMILY / DUPLEX']


[('COMMERCIAL',),
 ('MULTIFAMILY',),
 ('INDUSTRIAL',),
 ('INSTITUTIONAL',),
 ('SINGLE FAMILY / DUPLEX',)]

In [83]:
categories = ['All','COMMERCIAL','MULTIFAMILY', 'INDUSTRIAL','INSTITUTIONAL','SINGLE FAMILY / DUPLEX']


In [34]:
draw_markers.place_maps(test)

In [6]:
def myInteraction(selection):
    querry = str("""SELECT b.b_id as b_id, b.b_lat as b_latitude, b.b_long as b_longitude, count(*)/6 AS col_before, count(*)/3 AS col_during, count(*)/2 AS col_after 
    FROM buildings b, radius r 
    WHERE b.b_id = r.build_id AND """ + my_option +  "=" + "'" + my_selection + "'"  + 
    " GROUP BY b.b_id")
    crsr.execute(querry)
    test = pd.DataFrame(crsr.fetchall())
    test.columns = list(map(lambda x: x[0], crsr.description))
 
    

In [50]:
my_option = "b.b_category"
my_selection = "COMMERCIAL"
querry = str("SELECT b.b_id as b_id, b.b_lat as b_latitude, b.b_long as b_longitude, count(*)/6 AS col_before, count(*)/3 AS col_during, count(*)/2 AS col_after FROM buildings b, radius r WHERE b.b_id = r.build_id AND " + my_option +  "=" + "'" + my_selection + "'"  + " GROUP BY b.b_id")
querry
crsr.execute(querry)


In [52]:
querry = str("""SELECT b.b_id as b_id, b.b_lat as b_latitude, b.b_long as b_longitude, count(*)/6 AS col_before, count(*)/3 AS col_during, count(*)/2 AS col_after 
    FROM buildings b, radius r 
    WHERE b.b_id = r.build_id AND """ + my_option +  "=" + "'" + my_selection + "'"  + 
    " GROUP BY b.b_id")
crsr.execute(querry)